In [ ]:
import configobj
from crate import client
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt
import re
from teams import mlb_teams
team_keys = mlb_teams

In [ ]:
dfsub = pd.read_excel("../data/MLB_DB_2015.xlsx")

dfsub_dict = dfsub.to_dict('list')

home_team = []
away_team = []

for full_team, vs_team in zip(dfsub_dict['Team'],dfsub_dict['Vs']):

    if vs_team.split()[0]=='@':
        
        away_team.append(team_keys[full_team])
        home_team.append(vs_team.split()[1])
        
    else:
        
        away_team.append(vs_team)
        home_team.append(team_keys[full_team])
        
dfsub_dict["Home"]=home_team
dfsub_dict["Visitor"]=away_team

dfsub = pd.DataFrame(dfsub_dict)

print dfsub.head(100)

dfsub.to_csv("test.csv")

In [ ]:
games = []

nrows = len(dfsub.Season)

for i,row in enumerate(dfsub.iterrows()):
    if i%10.==0:
        print i, nrows
            
    for row2 in dfsub.iterrows():

        if row[1]['Date']==row2[1]['Date'] \
            and row[1]['Runs Scr']==row2[1]['Runs Alw'] \
            and row[1]['O/U Line']==row2[1]['O/U Line'] \
            and row[1]['Home']==row2[1]['Home'] \
            and row[1]['Visitor']==row2[1]['Visitor'] \
            and row[1]['FIELD']=='HOME':

                home_score = row[1]['Runs Scr']
                home_starter = row[1]['Home Starter']
                away_starter = row[1]['Away Starter']
                away_score = row[1]['Runs Alw']
                home_line = row[1]['Side Line']
                away_line = row2[1]['Side Line']
                over_under = row[1]['O/U Line']

                date = row[1]['Date']
                intdate = int(date.strftime('%Y%m%d'))

                games.append({'Home':row[1]['Home'],
                              'Away':row[1]['Visitor'],
                              'Date':intdate,
                              'Home Score':home_score,
                              'Away Score':away_score,
                              'Home Starter':home_starter,
                              'Away Starter':away_starter,
                              'Home Line':home_line,
                              'Away Line':away_line,
                              'Over Under':over_under})

In [ ]:
dfgames = pd.DataFrame(games)
dfgames.drop_duplicates(inplace='True')
print dfgames

In [ ]:
dictgames = dfgames.to_dict('records')
print len(dictgames)
print dictgames[0]

In [ ]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

for game in dictgames:
    
        game_id = str(game['Date']) + '_' + game['Away'] + '_' + game['Home']
        #check if the game is already in the databse
        query = '''SELECT * from mlb.games WHERE "GameId"='%s' '''%game_id

        cursor.execute(query)
        if len([row for row in cursor]) == 0:

            #games
            row = [game_id, game['Home'], game['Away'], game['Date'], game['Home Starter'], game['Away Starter']]
            query = '''INSERT INTO mlb.games ("GameId","HomeTeam", "AwayTeam", "GameDate", "HomeStarter", "AwayStarter") VALUES (?,?,?,?,?,?)'''
            cursor.execute(query, row)
            #lines
            row = [game_id, game['Home Line'], game['Away Line'], game['Over Under']]
            query = '''INSERT INTO mlb.lines ("GameId", "HomeLine", "AwayLine", "OverUnder")
                    VALUES (?,?,?,?)'''
            cursor.execute(query, row)
            #results
            row = [game_id, game['Home Score'], game['Away Score']]
            query = '''INSERT INTO mlb.results ("GameId", "HomeScore", "AwayScore")
                    VALUES (?,?,?)'''
            cursor.execute(query, row)

In [ ]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

pitcher_dataset = pd.read_csv("/Users/smacmullin/sports/data/MLB_pitchers_2014.csv")#,delimiter='\t')
print pitcher_dataset.head()

pitcher_dataset_dict = pitcher_dataset.to_dict("records")
for entry in pitcher_dataset_dict:

    pitcher = entry['Name'].decode('utf-8','ignore')
    pitcher = re.sub(r"(?<=\w)([A-Z])", r" \1", pitcher)

    wlpct = entry['W-L%'] # convert NaN to 0
    if np.isnan(entry['W-L%']):
        wlpct = 0.0
    era = entry['ERA']
    if np.isnan(entry['ERA']) or np.isinf(entry['ERA']):
        era = 0.0
        
    row = [2014, pitcher, entry['UniqueName'], entry['Tm'], wlpct, era, 
           entry['W'], entry['L'], entry['G'], entry['GS'], entry['GF'], entry['IP']]
    
    query = '''INSERT INTO mlb.pitchers ("Season","Name", "UniqueName", "Team", "WLPct", "ERA", "Wins", "Losses", "Games", "GamesStarted", "GamesFinished", "InningsPitched") VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'''
    cursor.execute(query, row)

# "Season" int,
# "Name" string,
# "UniqueName" string,
# "Team" string,
# "WLPct" float,
# "ERA" float,
# "Wins" int,
# "Losses" int,
# "Games" int,
# "GamesStarted" int,
# "GamesFinished" int,
# "InningsPitched" float

In [ ]:
pitching_dataset = pd.read_csv("/Users/smacmullin/sports/data/MLB_pitching_2014.csv")
print pitching_dataset.head()

pitching_dataset_dict = pitching_dataset.to_dict("records")
for entry in pitching_dataset_dict:
     
    row = [2014, entry['Tm'], entry['RAPG'], entry['ERA']]
    #print row
    query = '''INSERT INTO mlb.pitching ("Season", "Team", "RAPG", "ERA") VALUES (?,?,?,?)'''
    cursor.execute(query, row)
    
# "Season" int,
# "Team" string,
# "RAPG" float,
# "ERA" float,

In [ ]:
batting_dataset = pd.read_csv("/Users/smacmullin/sports/data/MLB_batting_2014.csv")
print batting_dataset.head()

batting_dataset_dict = batting_dataset.to_dict("records")
for entry in batting_dataset_dict:

    row = [2014, entry['Tm'], entry['R/G'], entry['OBP']]
    #print row
    query = '''INSERT INTO mlb.batting ("Season","Team", "RPG", "OBP") VALUES (?,?,?,?)'''
    cursor.execute(query, row)
    
# "Season" int,
# "Team" string,
# "RPG" float,
# "OBP" float,